# Auto Insurance Fraud Detection, Part 4 : Deploy, Run Inference, Interpret Inference

In this section of the end to end use case, we will deploy the mitigated model that is the end-product of this fraud detection use-case. We will show how to run inference and also how to use Clarify to interpret or "explain" the model.

<a id='overview-4'></a>

## [Overview](./0-AutoClaimFraudDetection.ipynb)
* ### [Notebook 1](./1-data-prep-e2e.ipynb)
  * #### [Getting started](#aud-getting-started)
  * #### [DataSets](#aud-datasets)
  * #### [SageMaker Feature Store](#aud-feature-store)
  * #### [Create train and test datasets](#aud-dataset)
* ### [Notebook 2](./2-lineage-train-assess-bias-tune-registry-e2e.ipynb)
  * #### [Train a model using XGBoost](#aud-train-model)
  * #### [Model lineage with artifacts and associations](#model-lineage)
  * #### [Evaluate the model for bias with Clarify](#check-bias)
  * #### [Deposit Model and Lineage in SageMaker Model Registry](#model-registry)
* ### [Notebook 3](./3-mitigate-bias-train-model2-registry-e2e.ipynb)
  * #### [Develop a second model](#second-model)
  * #### [Analyze the Second Model for Bias](#analyze-second-model)
  * #### [View Results of Clarify Bias Detection Job](#view-second-clarify-job)
  * #### [Configure and Run Clarify Explainability Job](#explainability)
  * #### [Create Model Package for second trained model](#model-package)
  
* ### [Notebook 4](./4-deploy-run-inference-e2e.ipynb)
  * #### [Architecture](#deploy)
  * #### [Deploy an approved model and Run Inference via Feature Store](#deploy-model)
  * #### [Create a Predictor](#predictor)
  * #### [Run  Predictions from Online FeatureStore](#run-predictions)
* ### [Notebook 5](./5-pipeline-e2e.ipynb)
  * #### SageMaker Pipeline
  * #### Cleanup

### Load stored variables
If you ran this notebook before, you may want to re-use the resources you aready created with AWS. Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything you may need to create them again or it may be your first time running this notebook.

In [9]:
%store -r
%store

Stored variables and their in-db values:
bucket                          -> 'sagemaker-us-east-2-738335684114'
claims_fg_name                  -> 'fraud-detect-demo-claims'
claims_table                    -> 'fraud-detect-demo-claims-1610061189'
col_order                       -> ['fraud', 'incident_type_theft', 'policy_state_ca'
customers_fg_name               -> 'fraud-detect-demo-customers'
customers_table                 -> 'fraud-detect-demo-customers-1610061192'
database_name                   -> 'sagemaker_featurestore'
hyperparameters                 -> {'max_depth': '3', 'eta': '0.2', 'objective': 'bin
model_1_name                    -> 'fraud-detect-demo-xgboost-pre-smote'
model_2_name                    -> 'fraud-detect-demo-xgboost-post-smote'
mp2_arn                         -> 'arn:aws:sagemaker:us-east-2:738335684114:model-pa
mpg_name                        -> 'fraud-detect-demo'
prefix                          -> 'fraud-detect-demo'
test_data_uri                   -> 's3

### Install required and/or update third-party libraries

In [10]:
!python -m pip install -Uq pip
!python -m pip install -q awswrangler==2.2.0 imbalanced-learn==0.7.0 sagemaker==2.23.1 boto3==1.16.48

### Import libraries

In [11]:
import json
import time
import boto3
import sagemaker
import numpy as np
import pandas as pd
import awswrangler as wr

### Set region, boto3 and SageMaker SDK variables

In [12]:
region = "us-east-2"

In [32]:
boto3.setup_default_session(region_name=region)

boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)

sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()

account_id = boto3.client('sts').get_caller_identity()["Account"]

In [33]:
# variables used for parameterizing the notebook run
endpoint_name = f'{model_2_name}-endpoint'
endpoint_instance_count = 1
endpoint_instance_type = "ml.m4.xlarge"

predictor_instance_count = 1
predictor_instance_type = "ml.c5.xlarge"
batch_transform_instance_count = 1
batch_transform_instance_type = "ml.c5.xlarge"

###  We assume you have run the previous data prep notebook,features are in the feature store, data is imported from them into an S3 bucket specified above as BUCKET and a train and test split has been done and put in the S3 bucket

<a id ='deploy'> </a>
### Architecture for this ML Lifecycle Stage : Train, Check Bias, Tune, Record Lineage, Register Model

![train-assess-tune-register](./images/e2e-3-pipeline-v3b.png)

<a id ='deploy-model'></a>

## Deploy an approved model and make prediction via Feature Store

[overview](#overview-4)

#### Approve second model
In the real-life MLOps lifecycle, a model package gets approved after evaluation by data scientists, subject matter experts and auditors.

In [73]:
second_model_package = sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]
model_package_update = {
    'ModelPackageArn': second_model_package['ModelPackageArn'],
    'ModelApprovalStatus': 'Approved'
}

update_response = sagemaker_boto_client.update_model_package(**model_package_update)

#### Create endpoint config and endpoint
Deploying the endpoint may take ~8min

In [74]:
primary_container = {'ModelPackageName': second_model_package['ModelPackageArn']}
endpoint_config_name=f'{model_2_name}-endpoint-config'
existing_configs = sagemaker_boto_client.list_endpoint_configs(NameContains=endpoint_config_name, MaxResults = 30)['EndpointConfigs'][0]['EndpointConfigName']

if not existing_configs:
    create_ep_config_response = sagemaker_boto_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': endpoint_instance_type,
            'InitialVariantWeight': 1,
            'InitialInstanceCount': endpoint_instance_count,
            'ModelName': model_2_name,
            'VariantName': 'AllTraffic'
        }]
    )
    %store endpoint_config_name





In [75]:
existing_endpoints = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name, MaxResults = 30)['Endpoints']
if not existing_endpoints:
    create_endpoint_response = sagemaker_boto_client.create_endpoint(
        EndpointName=endpoint_name,
        EndpointConfigName=endpoint_config_name)
    %store endpoint_name

endpoint_info = sagemaker_boto_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = endpoint_info['EndpointStatus']

while endpoint_status == 'Creating':
    endpoint_info = sagemaker_boto_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = endpoint_info['EndpointStatus']
    print('Endpoint status:', endpoint_status)
    if endpoint_status == 'Creating':
        time.sleep(60)

<a id='predictor'> </a>

### Create a predictor

In [60]:
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session)

### Sample a claim from the test data

In [61]:
dataset = pd.read_csv('data/dataset.csv')
train = dataset.sample(frac=0.8, random_state=0)
test = dataset.drop(train.index)
sample_policy_id  = int(test.sample(1)['policy_id'])

In [77]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 10 to 19994
Data columns (total 48 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       4000 non-null   int64  
 1   policy_id                        4000 non-null   int64  
 2   incident_type_theft              4000 non-null   int64  
 3   policy_state_ca                  4000 non-null   int64  
 4   policy_deductable                4000 non-null   int64  
 5   num_witnesses                    4000 non-null   int64  
 6   policy_state_or                  4000 non-null   int64  
 7   incident_month                   4000 non-null   int64  
 8   customer_gender_female           4000 non-null   int64  
 9   num_insurers_past_5_years        4000 non-null   int64  
 10  customer_gender_male             4000 non-null   int64  
 11  total_claim_amount               4000 non-null   float64
 12  authorities_contac

### Get sample's claim data from online feature store
This will simulate getting data in real-time from a customer's insurance claim submission.

In [78]:
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

<a id='run-predictions'> </a>
## Run Predictions on Multiple Claims

[overview](#overview-4)

In [79]:
import datetime  as datetime
timer =[]
MAXRECS = 100

def barrage_of_inference():
    sample_policy_id  = int(test.sample(1)['policy_id'])
    
    temp_fg_name = 'fraud-detect-demo-claims'

    claims_response = featurestore_runtime.get_record(
        FeatureGroupName=temp_fg_name, 
        RecordIdentifierValueAsString= str(sample_policy_id)

    )

    if (claims_response.get('Record')):
        claims_record = claims_response['Record']
        claims_df = pd.DataFrame(claims_record).set_index('FeatureName')
    else:
        print ("No Record returned / Record Key  \n")
        
    t0 = datetime.datetime.now()
    
    customers_response = featurestore_runtime.get_record(
        FeatureGroupName=customers_fg_name, 
        RecordIdentifierValueAsString=str(sample_policy_id)
    )
    
    t1 = datetime.datetime.now()

    customer_record = customers_response['Record']
    customer_df = pd.DataFrame(customer_record).set_index('FeatureName')
    
    
    blended_df = pd.concat([claims_df, customer_df]).loc[col_order].drop('fraud')
    data_input = ','.join(blended_df['ValueAsString'])
    
    results = predictor.predict(data_input, initial_args = {"ContentType": "text/csv"})
    prediction = json.loads(results)
    #print (f'Probablitity the claim from policy {int(sample_policy_id)} is fraudulent:', prediction)
    
    arr = t1-t0
    minutes, seconds = divmod(arr.total_seconds(), 60)
    
    timer.append(seconds)
    #print (prediction, " done in {} ".format(seconds))
    
    return sample_policy_id, prediction


for i in range(MAXRECS):
    sample_policy_id, prediction = barrage_of_inference()
    print (f'Probablitity the claim from policy {int(sample_policy_id)} is fraudulent:', prediction)

Probablitity the claim from policy 4227 is fraudulent: 0.02327040024101734
Probablitity the claim from policy 1356 is fraudulent: 0.01118443626910448
Probablitity the claim from policy 1731 is fraudulent: 0.003912070766091347
Probablitity the claim from policy 4944 is fraudulent: 0.045740846544504166
Probablitity the claim from policy 1187 is fraudulent: 0.003912070766091347
Probablitity the claim from policy 871 is fraudulent: 0.019201211631298065
Probablitity the claim from policy 218 is fraudulent: 0.07194169610738754
Probablitity the claim from policy 1463 is fraudulent: 0.02203090861439705
Probablitity the claim from policy 4442 is fraudulent: 0.017646918073296547
Probablitity the claim from policy 578 is fraudulent: 0.08237599581480026
Probablitity the claim from policy 2182 is fraudulent: 0.008739297278225422
Probablitity the claim from policy 4021 is fraudulent: 0.007248613052070141
Probablitity the claim from policy 3346 is fraudulent: 0.04129285365343094
Probablitity the clai

In [80]:
timer

[0.077019,
 0.018236,
 0.010011,
 0.008005,
 0.007163,
 0.006872,
 0.00804,
 0.007495,
 0.008866,
 0.007994,
 0.0075,
 0.007073,
 0.008033,
 0.007152,
 0.007621,
 0.007443,
 0.006649,
 0.007119,
 0.007337,
 0.007584,
 0.00646,
 0.006511,
 0.007164,
 0.006222,
 0.006938,
 0.00641,
 0.007141,
 0.00726,
 0.007458,
 0.007501,
 0.007825,
 0.007531,
 0.008141,
 0.007019,
 0.007374,
 0.00713,
 0.006917,
 0.007048,
 0.007414,
 0.006961,
 0.007019,
 0.007504,
 0.006098,
 0.00718,
 0.0067,
 0.006769,
 0.006372,
 0.006789,
 0.006601,
 0.007083,
 0.081883,
 0.008172,
 0.008337,
 0.008729,
 0.01084,
 0.007802,
 0.007802,
 0.008018,
 0.007442,
 0.007585,
 0.007348,
 0.007294,
 0.007045,
 0.00722,
 0.007495,
 0.007367,
 0.00769,
 0.007377,
 0.008347,
 0.00727,
 0.007091,
 0.007144,
 0.007761,
 0.007414,
 0.00746,
 0.007498,
 0.007593,
 0.009609,
 0.008017,
 0.008341,
 0.008218,
 0.008451,
 0.007196,
 0.007453,
 0.007275,
 0.007665,
 0.008648,
 0.023717,
 0.007462,
 0.007732,
 0.008362,
 0.007605,
 0.

Note: the above "timer" records the first call and then subsequent calls to the online Feature Store

In [81]:
import statistics
import numpy as np
statistics.mean(timer)  


arr = np.array(timer)
print("p95: {}, p99: {}, mean: {} for {} distinct feature store gets".format(np.percentile(arr,95),np.percentile(arr,99),np.mean(arr), MAXRECS))

p95: 0.01005245, p99: 0.07706764000000003, mean: 0.00928106 for 100 distinct feature store gets


### Pull customer data from Customers feature group
When a customer submits an insurance claim online for instant approval, the insurance company will need to pull customer-specific data from the online feature store to add to the claim data as input for a model prediction.

In [82]:
customers_response = featurestore_runtime.get_record(
    FeatureGroupName=customers_fg_name, 
    RecordIdentifierValueAsString=str(sample_policy_id))

customer_record = customers_response['Record']
customer_df = pd.DataFrame(customer_record).set_index('FeatureName')


claims_response = featurestore_runtime.get_record(
    FeatureGroupName=claims_fg_name, 
    RecordIdentifierValueAsString=str(sample_policy_id))

claims_record = claims_response['Record']
claims_df = pd.DataFrame(claims_record).set_index('FeatureName')



### Format the datapoint
The datapoint must match the exact input format as the model was trained--with all features in the correct order. In this example, the `col_order` variable was saved when you created the train and test datasets earlier in the guide.

In [83]:
blended_df = pd.concat([claims_df, customer_df]).loc[col_order].drop('fraud')
data_input = ','.join(blended_df['ValueAsString'])

### Make prediction

In [84]:
results = predictor.predict(data_input, initial_args = {"ContentType": "text/csv"})
prediction = json.loads(results)
print (f'Probablitity the claim from policy {int(sample_policy_id)} is fraudulent:', prediction)

Probablitity the claim from policy 1244 is fraudulent: 0.009145423769950867


<pre>
</pre>

<a id='aud-workflow-pipeline'></a>
## In the [Next Notebook](./5-pipeline-e2e.ipynb) we will show you how to put all this together in a SageMaker Pipeline
___
Now that as a Data Scientist, you've manually experimented with each step in our machine learning workflow, you can take certain steps to allow for faster model creation and deployment without sacrificing transparency and tracking via model lineage. In the next section you will create a pipeline which trains a new model on SageMaker, persists the model in SageMaker and then adds the model to the registry and deploys it as a SageMaker hosted endpoint.